# 🧪 Running GitLab CI Locally with `gitlab-runner`

The folloing example shows how to test a `.gitlab-ci.yml` pipeline job locally using `gitlab-runner` in shell mode. We'll generate a test script and CI config, then run it without needing a GitLab server.

# Install gitlab-runner: 
brew install gitlab-runner  
brew install gitlab-ci-local


## Running a GitLab CI Pipeline Locally 

In the next cell, you will run a **GitLab CI pipeline locally** using the tool  
`gitlab-ci-local`.

This executes the jobs defined in `.gitlab-ci.yml` **on your machine**, without
requiring a GitLab server or a registered runner. It is meant for **development,
testing, and learning**, not for production pipelines.

### What to expect when you run the cell

- A small Git repository is created locally
- A simple test (`pytest`) is executed via GitLab CI
- The job output is printed to the notebook
- The pipeline should finish successfully

You may see messages about a missing `origin` remote (for example:
“unable to retrieve default remote branch”).  
These are **normal and harmless** in a local-only setup and do not indicate
an error.

### Key idea

> CI jobs can run locally, even without a remote repository.  
> In real projects, pipelines are orchestrated by GitLab on a server.

If the final output shows **PASS** and the pipeline finishes successfully,
your local CI setup is working correctly.


In [1]:
import os
import subprocess
from pathlib import Path

# ------------------------------------------------------------------
# 1. Base path: local repository (where the notebook is)
# ------------------------------------------------------------------
repo_root = Path.cwd()
proj = repo_root / "gitlab_demo"
proj.mkdir(parents=True, exist_ok=True)

# ------------------------------------------------------------------
# 2. Environment
# ------------------------------------------------------------------
env = os.environ.copy()
env["SHELL"] = "/bin/bash"

# If gitlab-runner is in a custom bin path, uncomment:
# env["PATH"] = f"{repo_root}/bin:" + env["PATH"]

# ------------------------------------------------------------------
# 3. Optional cleanup
# ------------------------------------------------------------------
if (proj / ".git").exists():
    subprocess.run("rm -rf .git", shell=True, cwd=proj)

# ------------------------------------------------------------------
# 4. Test file
# ------------------------------------------------------------------
(proj / "test_sample.py").write_text(
"""\
def add(a, b):
    return a + b

def test_add():
    assert add(2, 2) == 4
"""
)

# ------------------------------------------------------------------
# 5. GitLab CI configuration
# ------------------------------------------------------------------
(proj / ".gitlab-ci.yml").write_text(
"""\
stages:
  - test

pytest:
  stage: test
  script:
    - python -m pip install --upgrade pip
    - pip install pytest
    - pytest
"""
)

# ------------------------------------------------------------------
# 6. Git repository
# ------------------------------------------------------------------
subprocess.run("git init", shell=True, cwd=proj, env=env)
subprocess.run("git branch -m main", shell=True, cwd=proj, env=env)
subprocess.run("git config user.name 'CI Tester'", shell=True, cwd=proj, env=env)
subprocess.run("git config user.email 'ci@test.local'", shell=True, cwd=proj, env=env)
subprocess.run("git add .", shell=True, cwd=proj, env=env)
subprocess.run("git commit -m 'initial commit'", shell=True, cwd=proj, env=env)

# ------------------------------------------------------------------
# 7. Run GitLab CI job locally (gitlab-ci-local)
# ------------------------------------------------------------------
print(f"\n🚀 Running GitLab CI job in {proj}\n")

result = subprocess.run(
    "gitlab-ci-local",
    shell=True,
    cwd=proj,
    env=env
)

# ------------------------------------------------------------------
# 8. Result
# ------------------------------------------------------------------
if result.returncode == 0:
    print("\n✅ GitLab CI job executed successfully.")
else:
    print("\n❌ GitLab CI job failed.")


Leeres Git-Repository in /Users/rpotthas/all/e-ai_ml2/course/code/code14/gitlab_demo/.git/ initialisiert
[main (Root-Commit) 054459f] initial commit
 4 files changed, 16 insertions(+)
 create mode 100644 .gitlab-ci-local/.gitignore
 create mode 100644 .gitlab-ci.yml
 create mode 100644 __pycache__/test_sample.cpython-312-pytest-9.0.2.pyc
 create mode 100644 test_sample.py

🚀 Running GitLab CI job in /Users/rpotthas/all/e-ai_ml2/course/code/code14/gitlab_demo



Unable to retrieve default remote branch, falling back to `main`.
  Command failed with exit code 128: git symbolic-ref --short refs/remotes/origin/HEAD
Schwerwiegend: ref refs/remotes/origin/HEAD is not a symbolic ref
Using fallback git remote data
  Command failed with exit code 2: git remote get-url origin
Fehler: Remote-Repository 'origin' nicht gefunden
parsing and downloads finished in 81 ms.
json schema validated in 80 ms


pytest starting shell (test)
pytest $ python -m pip install --upgrade pip
pytest > Requirement already satisfied: pip in /Users/rpotthas/ropy/lib/python3.12/site-packages (25.3)
pytest $ pip install pytest
pytest > Requirement already satisfied: pytest in /Users/rpotthas/ropy/lib/python3.12/site-packages (9.0.2)
pytest > Requirement already satisfied: iniconfig>=1.0.1 in /Users/rpotthas/ropy/lib/python3.12/site-packages (from pytest) (2.3.0)
pytest > Requirement already satisfied: packaging>=22 in /Users/rpotthas/ropy/lib/python3.12/site-packages (from pytest) (25.0)
pytest > Requirement already satisfied: pluggy<2,>=1.5 in /Users/rpotthas/ropy/lib/python3.12/site-packages (from pytest) (1.6.0)
pytest > Requirement already satisfied: pygments>=2.7.2 in /Users/rpotthas/ropy/lib/python3.12/site-packages (from pytest) (2.19.2)
pytest $ pytest
pytest > ============================= test session starts ==============================
pytest > platform darwin -- Python 3.12.12, pytest-9.0.2, 

pipeline finished in 6.37 s
